# Paired a x b cross table
Alternative of z-test and chi-square test

In [ ]:
# Enable the commands below when running this program on Google Colab.
# !pip install arviz==0.7
# !pip install pymc3==3.8
# !pip install Theano==1.0.4

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import pymc3 as pm

import math

plt.style.use('seaborn-darkgrid')
np.set_printoptions(precision=3)
pd.set_option('display.precision', 3)

## Q. A restaurant counted up what wines (red, rose, and white) customers chose for their main dishes (roast veal, pasta gorgonzola, and sole meuniere). Analyze the relationship between main dish and wine.

In [ ]:
a = 3  # Num. of man dishes
b = 3  # Num. of wines
data = pd.DataFrame([[19, 12, 6], [8, 8, 4], [15, 19, 18]], columns=['Veal', 'Pasta', 'Sole'], index=['Red', 'Rose', 'White'])
observed = [data['Veal']['Red'], 
            data['Pasta']['Red'], 
            data['Sole']['Red'],
            data['Veal']['Rose'], 
            data['Pasta']['Rose'], 
            data['Sole']['Rose'],
            data['Veal']['White'], 
            data['Pasta']['White'], 
            data['Sole']['White']]
display(data)
N = data.sum().sum()

In [ ]:
with pm.Model() as model:
  # Prior distribution
  p_ = pm.Uniform('p_', 0, 1, shape=(a * b))
  p = pm.Deterministic('p', p_ / pm.math.sum(p_))

  # Likelihood
  x = pm.Multinomial('x', n=N, p=p, observed=observed)

  # Marginal probability
  p1d = pm.Deterministic('p1d', p[0] + p[1] + p[2])  # p1. = p11 + p12 + p13
  p2d = pm.Deterministic('p2d', p[3] + p[4] + p[5])  # p2. = p21 + p22 + p23
  p3d = pm.Deterministic('p3d', p[6] + p[7] + p[8])  # p3. = p31 + p32 + p33

  pd1 = pm.Deterministic('pd1', p[0] + p[3] + p[6])  # p.1 = p11 + p21 + p31
  pd2 = pm.Deterministic('pd2', p[1] + p[4] + p[7])  # p.2 = p12 + p22 + p32
  pd3 = pm.Deterministic('pd3', p[2] + p[5] + p[8])  # p.3 = p13 + p23 + p33

  # Pearson's residual
  pp = [p1d * pd1, p1d * pd2, p1d * pd3, 
        p2d * pd1, p2d * pd2, p2d * pd3, 
        p3d * pd1, p3d * pd2, p3d * pd3]
  e = pm.Deterministic('e', (p - pp) / pm.math.sqrt(pp))

  # Cramer's association coefficient
  V = pm.Deterministic('V', pm.math.sqrt(pm.math.sum(e**2) / (min(a, b) - 1)))

  trace = pm.sample(21000, chains=5)

In [ ]:
chain = trace[1000:]
pm.traceplot(chain)
plt.show()

In [ ]:
pm.summary(chain, var_names=['p', 'V', 'p1d', 'p2d', 'p3d', 'pd1', 'pd2', 'pd3'])

### Independence and association

In [ ]:
plt.boxplot(
    [chain['e'][:,0],
     chain['e'][:,1],
     chain['e'][:,2],
     chain['e'][:,3],
     chain['e'][:,4],
     chain['e'][:,5],
     chain['e'][:,6],
     chain['e'][:,7],
     chain['e'][:,8],],
     labels=['e11', 'e12', 'e13', 'e21', 'e22', 'e23', 'e31', 'e32', 'e33'])
plt.show()

In [ ]:
egz = pd.DataFrame(
    [[(chain['e'][:,0] > 0).mean(), (chain['e'][:,1] > 0).mean(), (chain['e'][:,2] > 0).mean()],
     [(chain['e'][:,3] > 0).mean(), (chain['e'][:,4] > 0).mean(), (chain['e'][:,5] > 0).mean()],
     [(chain['e'][:,6] > 0).mean(), (chain['e'][:,7] > 0).mean(), (chain['e'][:,8] > 0).mean()]
    ],
    columns=['Veal', 'Pasta', 'Sole'],
    index=['Red', 'Rose', 'White']
)

elz = pd.DataFrame(
    [[(chain['e'][:,0] < 0).mean(), (chain['e'][:,1] < 0).mean(), (chain['e'][:,2] < 0).mean()],
     [(chain['e'][:,3] < 0).mean(), (chain['e'][:,4] < 0).mean(), (chain['e'][:,5] < 0).mean()],
     [(chain['e'][:,6] < 0).mean(), (chain['e'][:,7] < 0).mean(), (chain['e'][:,8] < 0).mean()]
    ],
    columns=['Veal', 'Pasta', 'Sole'],
    index=['Red', 'Rose', 'White']
)

print('e > 0')
display(egz)
print('e < 0')
display(elz)

### RQ1: 「子牛」料理を選んだ客は「赤」を選び「白」は避け、「舌平目」料理を選んだ客は「白」を選び「赤」は避ける

In [ ]:
val_1 = (chain['e'][:,0] > 0).mean() * (chain['e'][:,8] > 0).mean() * (chain['e'][:,6] < 0).mean() * (chain['e'][:,2] < 0).mean()
print('Probability: {:.3f} %'.format(val_1 * 100))

### RQ2: 「子牛」料理を選んだ客は「赤」を選び「白」は避け、「舌平目」料理を選んだ客は「白」を選ぶ

In [ ]:
val_2 = (chain['e'][:,0] > 0).mean() * (chain['e'][:,8] > 0).mean() * (chain['e'][:,6] < 0).mean()
print('Probability: {:.3f} %'.format(val_2 * 100))

### RQ3: 「子牛」料理を選んだ客は「赤」を選び、「舌平目」料理を選んだ客は「白」を選ぶ

In [ ]:
val_3 = (chain['e'][:,0] > 0).mean() * (chain['e'][:,8] > 0).mean()
print('Probability: {:.3f} %'.format(val_3 * 100))